<a href="https://colab.research.google.com/github/Nishi2065s/Python/blob/main/passwordmanager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Nishi2065s/security-.git

In [ ]:
!pip install bcrypt

In [ ]:
!apt-get install -y hashcat


In [ ]:
!hashcat --version


In [ ]:
!ls /usr/share/wordlists/

In [ ]:
!apt-get install -y wordlists
!gunzip /usr/share/wordlists/rockyou.txt.gz

In [ ]:
!wget https://github.com/brannondorsey/naive-hashcat/releases/download/data/rockyou.txt


In [ ]:
!mkdir -p /usr/share/wordlists
!mv rockyou.txt /usr/share/wordlists/

In [ ]:
import hashlib
import bcrypt
import time
import random
import multiprocessing
from hashlib import sha256
from itertools import product
import string

In [ ]:
import hashlib
import time
import subprocess
import os
from hashlib import sha256
import random
import string

# Blockchain class for digital signature authentication
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(previous_hash='0')  # Genesis block

    def create_block(self, previous_hash, data=None):
        block = {
            'index': len(self.chain) + 1,
            'data': data,
            'previous_hash': previous_hash,
            'hash': self.hash_block(previous_hash + (data or ''))
        }
        self.chain.append(block)
        return block

    def hash_block(self, data):
        return sha256(data.encode()).hexdigest()

    def get_last_block(self):
        return self.chain[-1] if self.chain else None

# Use Hashcat for password cracking
def crack_with_hashcat(password, time_limit=240):
    start_time = time.time()
    hash_file = "hashes.txt"
    output_file = "cracked.txt"

    hashed_password = sha256(password.encode()).hexdigest()

    with open(hash_file, "w") as f:
        f.write(f"{hashed_password}\n")

    command = [
        "hashcat", "-m", "1400", hash_file, "/usr/share/wordlists/rockyou.txt", "--force",
        "--outfile", output_file, "--runtime", str(time_limit), "--attack-mode=0"
    ]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
        print(result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print("Error running Hashcat:", e.stderr.decode())
        return None, time.time() - start_time

    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        with open(output_file, "r") as f:
            cracked_password = f.readline().strip().split(':')[-1]
            return cracked_password, time.time() - start_time
    return None, time.time() - start_time

# Password Manager Class
class PasswordManager:
    def __init__(self):
        self.blockchain = Blockchain()

    def derive_signature(self, password):
        return sha256(password.encode()).hexdigest()

    def test_password_strength(self, password):
        print("Testing password strength...")
        cracked_password, time_taken = crack_with_hashcat(password)

        if cracked_password:
            print(f"Weak Password! Cracked in {time_taken:.2f} seconds. Reset your password.")
            return False
        else:
            print("Strong Password! Not cracked within 4 minutes.")
            return True

    def authenticate(self, password):
        user_signature = self.derive_signature(password)
        last_block = self.blockchain.get_last_block()

        if last_block and last_block['data'] == user_signature:
            print("Authentication successful!")
            return True
        print("Authentication failed!")
        return False

    def store_user(self, username, password):
        user_signature = self.derive_signature(password)

        print(f"Password stored securely for {username}.")
        print(f"Generated Digital Signature: {user_signature}")

        # Store signature in blockchain
        self.blockchain.create_block(previous_hash=self.blockchain.get_last_block()['hash'], data=user_signature)
        print("Digital Signature recorded in Blockchain.")

        return user_signature

# Testing the system
if __name__ == "__main__":
    pm = PasswordManager()

    username = input("Enter username: ").strip()
    password = input("Enter password: ").strip()

    if pm.test_password_strength(password):
        _ = pm.store_user(username, password)
        auth_attempt = input("Enter your password to authenticate: ").strip()
        pm.authenticate(auth_attempt)